In [8]:
import mlflow.pyfunc
import mlflow.keras
import os
os.environ["AWS_ACCESS_KEY_ID"] = "mlflow"
os.environ["AWS_SECRET_ACCESS_KEY"] = "mlflow123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://mlflow-minio.mlflow.svc.cluster.local:9000/"
mlflow.set_tracking_uri("http://mlflow.mlflow.svc.cluster.local")
mlflow.set_experiment("bitcoin")

model_name = "BitcoinForecast"
model_version = 1

model = mlflow.keras.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)


# Get Model

In [24]:
from mlflow import MlflowClient

client = MlflowClient()
for rm in client.search_registered_models(filter_string="name = 'BitcoinForecast'"):
    print(rm)


<RegisteredModel: creation_timestamp=1664524385958, description='', last_updated_timestamp=1664524386014, latest_versions=[<ModelVersion: creation_timestamp=1664524386014, current_stage='None', description='', last_updated_timestamp=1664524386014, name='BitcoinForecast', run_id='1a5d699480ab46ab9c31d28c3550ffa0', run_link='', source='s3://mlflow/1/1a5d699480ab46ab9c31d28c3550ffa0/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>], name='BitcoinForecast', tags={}>


# Get Run

In [36]:
rm.latest_versions[-1].run_id

'1a5d699480ab46ab9c31d28c3550ffa0'

In [39]:
run = client.get_run(rm.latest_versions[0].run_id)

# Get Params

In [58]:
params = run.data.params

In [64]:
!pwd

/home/pierre/Realtime-MLOps/platform/training


In [67]:
from feast import FeatureStore
feature_store = FeatureStore('../../demonstration/feature_store/feature_repo')  # Initialize the feature store

feature_service = feature_store.get_feature_service("crypto_stats")
feature_store.get_historical_features(features=feature_service, entity_df=entity_df)

NameError: name 'entity_df' is not defined

datetime.datetime(2022, 3, 1, 4, 40)

In [ ]:
pd.date_range(params["data_to"], datetime.now()

In [144]:
entity_df = pd.DataFrame.from_dict(
    {
        "symbol": "BTC/USD",
        "event_timestamp": [item for item in pd.date_range(datetime.fromtimestamp(float(params["data_to"])), datetime.now(), freq='5min')]
    }
)

In [145]:
entity_df

,symbol,event_timestamp
0,BTC/USD,2022-03-01 04:40:00
1,BTC/USD,2022-03-01 04:45:00
2,BTC/USD,2022-03-01 04:50:00
3,BTC/USD,2022-03-01 04:55:00
4,BTC/USD,2022-03-01 05:00:00
...,...,...
61450,BTC/USD,2022-09-30 13:30:00
61451,BTC/USD,2022-09-30 13:35:00
61452,BTC/USD,2022-09-30 13:40:00
61453,BTC/USD,2022-09-30 13:45:00


In [146]:
import pandas as pd
from datetime import datetime

#entity_sql = f"""
#    SELECT
#        symbol,
#        timestamp
#    FROM {feature_store.get_data_source("crypto_source").get_table_query_string()}
#    WHERE timestamp WHERE timestamp > TO_TIMESTAMP(1646106000.0))
#"""

training_df = feature_store.get_historical_features(
    entity_df=entity_df,
    features=feature_store.get_feature_service("crypto_stats")
)

In [149]:
df = training_df.to_df()
df = df.dropna()
first = df.index[0].timestamp()
last = df.index[-1].timestamp()

In [159]:
df['y'] = df['close'].shift(-1)
df = df[:-1]

In [161]:
df = df[["open" ,"high", "low", "close", "y"]].drop_duplicates() 

In [162]:
df

,open,high,low,close,y
0,43046.58,43046.58,43046.58,43046.58,19583.12
61384,19585.50,19592.66,19551.87,19583.12,19594.51
61385,19578.09,19597.23,19558.89,19594.51,19555.99
61386,19586.98,19600.10,19555.99,19555.99,19552.24
61387,19574.98,19574.98,19546.56,19552.24,19578.24
61388,19559.32,19586.20,19550.35,19578.24,19553.80
61389,19572.75,19572.75,19553.80,19553.80,19563.16
61390,19547.56,19565.01,19537.45,19563.16,19554.68
61391,19561.98,19561.98,19551.29,19554.68,19538.52
61392,19557.35,19557.35,19538.52,19538.52,19529.05


In [155]:
from sklearn.model_selection import train_test_split
random_state = 42
train_size = 0.8
X = df[['open', 'high', 'low', 'close']]
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)

KeyError: 'y'